# Gradient Descent and its Variants

This notebook is designed to write and explore multiple gradient descent algorithms.

### Setup

In [1]:
# Sets up directories and files for a new project
from pathlib import Path

cwd = Path.cwd()
graph_dir = cwd / 'Graphs'

def CreateDir(directory: Path) -> None:
    print(f'Creating directory ...')
    Path.mkdir(directory, exist_ok=True, parents=True)
    print(f'Directory {directory} created.')
    
def CreateFilePath(fname: str, dir: str = graph_dir,
                    extension: str = 'png') -> str:
    return f'{dir}{'\\'}{fname}.{extension}'

CreateDir(graph_dir)

Creating directory ...
Directory c:\Users\giaco\Documents\GitHub\ML-Practice\Notebooks\Gradient-Descent\Graphs created.


In [2]:
# Sets up plotting and other packages
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Any, Tuple, Optional
from numpy.typing import NDArray

# Changes the default font size, linewidth, and figure size
plt.rcParams.update({'font.size': 14,
                        'lines.linewidth': 2,
                        'figure.figsize': (10, 8),
                        'figure.titlesize': 18,
                        'axes.labelsize': 16,
                        'axes.titlesize': 18,
                        'axes.linewidth': 2,
                        'xtick.labelsize': 14,
                        'ytick.labelsize': 14,
                        'legend.fontsize': 14,
                        'legend.title_fontsize': 16,
                        'legend.shadow': True,
                        'legend.frameon': True,
                        'legend.edgecolor': 'black',
                        'legend.facecolor': 'white',
                        'legend.fancybox': True,
                        'legend.framealpha': 1,
                        'legend.borderpad': 1,
                        'legend.labelspacing': 1,
                        'legend.handletextpad': 1,
                        'legend.handlelength': 1.5,
                        'legend.borderaxespad': 1,
                        'legend.columnspacing': 2})

## Helper Functions

In [3]:
def ReshapeTo(arr: NDArray[Any],
                required_shape: Tuple[int, ...]) -> Optional[NDArray[Any]]:
    """Reshapes array to required shape. If current shape matches original array,
        does not reshape. If array cannot be reshaped to required shape

    Args:
        arr (NDArray[Any]): Array to be reshaped
        required_shape (Tuple[int, ...]): Array shape to which arr needs to be reshaped to

    Returns:
        Optional[NDArray[Any]]: Reshaped array
    """
    
    # Checks whether array has already required shape
    if arr.shape == required_shape:
        return arr
    
    # If not required shape, try to reshape
    try:
        reshaped_arr = arr.reshape(required_shape)
        return reshaped_arr
    
    except ValueError as e:
        print(f"Cannot reshape array from shape {arr.shape} to {required_shape}: {e}")
        return None
        
    return 

## Polynomial Model and Cost Function

In [13]:
def FormPredictorMatrix(predictors: NDArray[Any], order: int=1) -> NDArray[Any]:
    
    # Create array to store predictor matrix
    predictor_matrix = np.ones((predictors.shape[0], 1))
    for exp in range(1, order+1):
        predictor_matrix = np.hstack((predictor_matrix, predictors**exp))
        
    return predictor_matrix

def PolynomialModel(predictors_matrix: NDArray[Any],
                    params: NDArray[Any]) -> NDArray[Any]:
    
    """ Function to compute the model response for a polynomial model.
        Note: For n-th order polynomial, the predictors should be in the form of [X, X^2, X^3, ... X^n]

        Args:
            predictors_matrix (NDArray[Any]): Array of predictors value for each measurement 
                                        i.e. [X1, X2, X3, ...] where Xn is the column vector
                                        of measurements for nth predictor
            params (NDArray[Any]): Array of parameters for the model. 
                                    The first element is the intercept and the rest are the coefficients
                                    
        Returns:
            NDArray[Any]: Model response for the given predictors and parameters
        """
    
    # Asserts correct shape of predictors
    assert predictors_matrix.ndim<=2, "The predictors should be a 1-D Array containg multiple meausurements for one predictor or 2-D Array containing multiple meausurements for various predictors."
    assert params.ndim<=2 and params.shape[0]==predictors_matrix.shape[-1], "Parameter array must be a 1-D array (Row vector) or a 2-D Array with one column with the same number of entries as the number of columns of predictor matrix"
    
    # Reshapes param array
    params = ReshapeTo(arr=params, required_shape=(predictors_matrix.shape[-1], 1))
    
    return np.matmul(predictors_matrix, params)

def MeanSquareError(model: Callable[[NDArray[Any], NDArray[Any]], NDArray[Any]],
                    response: NDArray[Any],
                    predictors: list[NDArray[Any]],
                    params: NDArray[Any]) -> NDArray[Any]:
    
    assert response.ndim==1, "The response values should be a 1-D array"
    assert predictors.shape[-1]==params.shape[0]
    
    difference = (response - model(predictors, params)) #Computes difference between measured and model response as 1-D array 
    mean_squared_error = (difference).dot(difference)   #Squares error
    mean_squared_error /= len(response)                 #Takes average
    
    return mean_squared_error

## Gradient Descent Algorithms

In [ ]:
def GD(func: function, params: list[np.ndarray], init: np.ndarray, eta: float, epochs: int) -> np.ndarray:
    
    func_value = func(*params.T)                           #Unpacks Parameters Along columns
    trajectory = np.zeros(shape=(epochs+1, init.shape[0])) #Empty array to store trajectory in parameter space
    trajectory[0, :] = init                                #Initial position in parameter space
    
    #Itrates over epochs to update parameter trajectory
    for iter in range(epochs):
        grad = np.grad(func_value)
        trajectory[iter+1, :] = trajectory[iter]
        trajectory[iter+1, :] -= eta*grad
    
    return trajectory

In [ ]:
x = np.linspace(1, 10, 10)
y = np.linspace(1, 5, 5)
X, Y = np.meshgrid(x, y)
print(f"{X}{'\n\n'}{Y}")

[[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]]

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]]
(5,)
